## **Data Science 112 Final Project: Data Extraction** **bold text**


# Tricks for Tips : A Data-Driven Analysis of Tipping Influences
***by Amelie and Spurti***

This project explores variables that impact the tipping percentages. This section outlines the initial data import and preprocessing steps taken to build a unified, analyzable dataset. We imported over 100 million trip records from the NYC Taxi & Limousine Commission, supplemented with hourly weather data via API, ZIP-code level income data from web scraping, and geospatial mappings of taxi zones. Extensive cleaning, filtering, and merging were performed to ensure temporal, spatial, and contextual alignment across all data sources.


In [1]:
import gc
import random
import pandas as pd
from google.colab import drive
import pyarrow.parquet as pq
drive.mount('/content/drive')

monthly_dfs = []

for month in range(1,13):
  month_str = str(month).zfill(2)
  trips_month= pq.read_table('/content/drive/My Drive/yellow_tripdata_2019-' + month_str + '.parquet').to_pandas()
  trips_month_sample = trips_month.sample(n=1000, random_state=42)
  monthly_dfs.append(trips_month_sample)
  del trips_month, trips_month_sample
  gc.collect()

df = pd.concat(monthly_dfs, ignore_index=True)
df

Mounted at /content/drive


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2019-01-09 22:54:47,2019-01-09 23:23:09,1.0,6.20,1.0,N,249,179,1,23.0,0.5,0.5,2.00,0.0,0.3,26.30,NaN,None
1,2,2019-01-11 21:30:16,2019-01-11 21:53:14,2.0,4.55,1.0,N,114,142,2,19.5,0.5,0.5,0.00,0.0,0.3,20.80,NaN,None
2,1,2019-01-06 19:06:07,2019-01-06 19:23:40,1.0,2.60,1.0,N,163,249,1,13.0,0.0,0.5,2.50,0.0,0.3,16.30,NaN,None
3,4,2019-01-12 04:23:27,2019-01-12 04:39:41,1.0,4.37,1.0,N,234,256,1,16.0,0.5,0.5,3.46,0.0,0.3,20.76,NaN,None
4,2,2019-01-31 11:46:57,2019-01-31 11:55:58,1.0,0.89,1.0,N,90,246,1,7.5,0.0,0.5,1.66,0.0,0.3,9.96,0.0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11995,2,2019-12-22 21:15:14,2019-12-22 21:20:21,2.0,0.77,1.0,N,113,107,1,5.5,0.5,0.5,1.40,0.0,0.3,10.70,2.5,None
11996,1,2019-12-02 08:48:54,2019-12-02 09:00:47,1.0,2.20,1.0,N,52,106,1,10.5,0.0,0.5,1.00,0.0,0.3,12.30,0.0,None
11997,1,2019-12-03 16:01:29,2019-12-03 16:09:51,1.0,1.20,1.0,N,170,107,1,7.5,3.5,0.5,1.20,0.0,0.3,13.00,2.5,None
11998,2,2019-12-24 13:17:01,2019-12-24 13:19:07,1.0,0.48,1.0,N,90,90,1,3.5,0.0,0.5,1.36,0.0,0.3,8.16,2.5,None


In [2]:
df = df.drop(['VendorID', 'extra', 'mta_tax', 'tolls_amount', 'improvement_surcharge', 'store_and_fwd_flag'], axis=1)

In [3]:
df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])
df['pickup_date'] = df['tpep_pickup_datetime'].dt.date
df['pickup_time'] = df['tpep_pickup_datetime'].dt.time
df['dropoff_time'] = df['tpep_dropoff_datetime'].dt.time
df['day_of_week'] = df['tpep_pickup_datetime'].dt.day_name()
df['trip_duration'] = (df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']).dt.total_seconds() / 60

df["congestion_surcharge"].fillna(0)
df["airport_fee"] = pd.to_numeric(df["airport_fee"], errors='coerce')
df["airport_fee"] = df["airport_fee"].fillna(0).astype(int)
trips_df = df
trips_df

,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount,tip_amount,total_amount,congestion_surcharge,airport_fee,pickup_date,pickup_time,dropoff_time,day_of_week,trip_duration
0,2019-01-09 22:54:47,2019-01-09 23:23:09,1.0,6.20,1.0,249,179,1,23.0,2.00,26.30,NaN,0,2019-01-09,22:54:47,23:23:09,Wednesday,28.366667
1,2019-01-11 21:30:16,2019-01-11 21:53:14,2.0,4.55,1.0,114,142,2,19.5,0.00,20.80,NaN,0,2019-01-11,21:30:16,21:53:14,Friday,22.966667
2,2019-01-06 19:06:07,2019-01-06 19:23:40,1.0,2.60,1.0,163,249,1,13.0,2.50,16.30,NaN,0,2019-01-06,19:06:07,19:23:40,Sunday,17.550000
3,2019-01-12 04:23:27,2019-01-12 04:39:41,1.0,4.37,1.0,234,256,1,16.0,3.46,20.76,NaN,0,2019-01-12,04:23:27,04:39:41,Saturday,16.233333
4,2019-01-31 11:46:57,2019-01-31 11:55:58,1.0,0.89,1.0,90,246,1,7.5,1.66,9.96,0.0,0,2019-01-31,11:46:57,11:55:58,Thursday,9.016667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11995,2019-12-22 21:15:14,2019-12-22 21:20:21,2.0,0.77,1.0,113,107,1,5.5,1.40,10.70,2.5,0,2019-12-22,21:15:14,21:20:21,Sunday,5.116667
11996,2019-12-02 08:48:54,2019-12-02 09:00:47,1.0,2.20,1.0,52,106,1,10.5,1.00,12.30,0.0,0,2019-12-02,08:48:54,09:00:47,Monday,11.883333
11997,2019-12-03 16:01:29,2019-12-03 16:09:51,1.0,1.20,1.0,170,107,1,7.5,1.20,13.00,2.5,0,2019-12-03,16:01:29,16:09:51,Tuesday,8.366667
11998,2019-12-24 13:17:01,2019-12-24 13:19:07,1.0,0.48,1.0,90,90,1,3.5,1.36,8.16,2.5,0,2019-12-24,13:17:01,13:19:07,Tuesday,2.100000


In [4]:
!pip install openmeteo-requests
!pip install requests-cache retry-requests numpy pandas

import openmeteo_requests

import pandas as pd
import requests_cache
from retry_requests import retry

cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": 52.52,
	"longitude": 13.41,
	"start_date": "2019-01-01",
	"end_date": "2019-12-31",
	"hourly": ["temperature_2m", "rain", "precipitation", "wind_speed_10m", "pressure_msl", "snowfall", "snow_depth", "weather_code"]
}
responses = openmeteo.weather_api(url, params=params)

response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()}{response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_rain = hourly.Variables(1).ValuesAsNumpy()
hourly_precipitation = hourly.Variables(2).ValuesAsNumpy()
hourly_wind_speed_10m = hourly.Variables(3).ValuesAsNumpy()
hourly_pressure_msl = hourly.Variables(4).ValuesAsNumpy()
hourly_snowfall = hourly.Variables(5).ValuesAsNumpy()
hourly_snow_depth = hourly.Variables(6).ValuesAsNumpy()
hourly_weather_code = hourly.Variables(7).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}

hourly_data["temperature_2m"] = hourly_temperature_2m
hourly_data["rain"] = hourly_rain
hourly_data["precipitation"] = hourly_precipitation
hourly_data["wind_speed_10m"] = hourly_wind_speed_10m
hourly_data["pressure_msl"] = hourly_pressure_msl
hourly_data["snowfall"] = hourly_snowfall
hourly_data["snow_depth"] = hourly_snow_depth
hourly_data["weather_code"] = hourly_weather_code

hourly_dataframe = pd.DataFrame(data = hourly_data)
weather_df= hourly_dataframe
weather_df

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.1/165.1 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 668.4/668.4 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.9/247.9 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.4/393.4 kB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 65.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.4/69.4 kB 5.1 MB/s eta 0:00:00
Coordinates 52.5483283996582°N 13.407821655273438°E
Elevation 38.0 m asl
Timezone NoneNone
Timezone difference to GMT+0 0 s


,date,temperature_2m,rain,precipitation,wind_speed_10m,pressure_msl,snowfall,snow_depth,weather_code
0,2019-01-01 00:00:00+00:00,7.4085,0.0,0.0,18.861387,1025.500000,0.0,0.0,3.0
1,2019-01-01 01:00:00+00:00,7.5085,0.0,0.0,21.659918,1024.699951,0.0,0.0,3.0
2,2019-01-01 02:00:00+00:00,7.2585,0.0,0.0,23.688984,1023.900024,0.0,0.0,3.0
3,2019-01-01 03:00:00+00:00,7.2085,0.0,0.0,26.867676,1022.299988,0.0,0.0,3.0
4,2019-01-01 04:00:00+00:00,7.2585,0.1,0.1,30.312773,1020.900024,0.0,0.0,51.0
...,...,...,...,...,...,...,...,...,...
8755,2019-12-31 19:00:00+00:00,3.6585,0.0,0.0,14.658676,1032.099976,0.0,0.0,0.0
8756,2019-12-31 20:00:00+00:00,3.1585,0.0,0.0,12.904882,1032.300049,0.0,0.0,0.0
8757,2019-12-31 21:00:00+00:00,2.7585,0.0,0.0,13.104198,1032.800049,0.0,0.0,0.0
8758,2019-12-31 22:00:00+00:00,2.2585,0.0,0.0,11.966954,1033.000000,0.0,0.0,0.0


In [5]:
trips_df['pickup_hour'] = pd.to_datetime(trips_df['tpep_pickup_datetime']).dt.floor('h')
weather_df['weather_time'] = pd.to_datetime(weather_df['date'], utc=True)
weather_df['weather_time'] = weather_df['weather_time'].dt.tz_convert(None)
merged_df = pd.merge(df, weather_df, left_on='pickup_hour', right_on='weather_time', how='left').drop(['weather_time', 'date', 'RatecodeID', 'pickup_hour', 'weather_code'], axis =1)
merged_df

,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,PULocationID,DOLocationID,payment_type,fare_amount,tip_amount,total_amount,...,dropoff_time,day_of_week,trip_duration,temperature_2m,rain,precipitation,wind_speed_10m,pressure_msl,snowfall,snow_depth
0,2019-01-09 22:54:47,2019-01-09 23:23:09,1.0,6.20,249,179,1,23.0,2.00,26.30,...,23:23:09,Wednesday,28.366667,2.5085,0.1,0.1,23.110207,1015.500000,0.0,0.01
1,2019-01-11 21:30:16,2019-01-11 21:53:14,2.0,4.55,114,142,2,19.5,0.00,20.80,...,21:53:14,Friday,22.966667,4.5585,0.0,0.0,20.858265,1012.500000,0.0,0.01
2,2019-01-06 19:06:07,2019-01-06 19:23:40,1.0,2.60,163,249,1,13.0,2.50,16.30,...,19:23:40,Sunday,17.550000,1.1085,0.0,0.0,5.447788,1029.400024,0.0,0.00
3,2019-01-12 04:23:27,2019-01-12 04:39:41,1.0,4.37,234,256,1,16.0,3.46,20.76,...,04:39:41,Saturday,16.233333,2.2585,0.0,0.0,16.735161,1014.299988,0.0,0.01
4,2019-01-31 11:46:57,2019-01-31 11:55:58,1.0,0.89,90,246,1,7.5,1.66,9.96,...,11:55:58,Thursday,9.016667,2.6585,0.0,0.0,7.568566,1001.200012,0.0,0.01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11995,2019-12-22 21:15:14,2019-12-22 21:20:21,2.0,0.77,113,107,1,5.5,1.40,10.70,...,21:20:21,Sunday,5.116667,5.1085,0.0,0.0,12.768586,990.799988,0.0,0.00
11996,2019-12-02 08:48:54,2019-12-02 09:00:47,1.0,2.20,52,106,1,10.5,1.00,12.30,...,09:00:47,Monday,11.883333,2.1085,0.0,0.0,21.252199,1018.700012,0.0,0.00
11997,2019-12-03 16:01:29,2019-12-03 16:09:51,1.0,1.20,170,107,1,7.5,1.20,13.00,...,16:09:51,Tuesday,8.366667,4.3585,0.0,0.0,12.496718,1025.400024,0.0,0.00
11998,2019-12-24 13:17:01,2019-12-24 13:19:07,1.0,0.48,90,90,1,3.5,1.36,8.16,...,13:19:07,Tuesday,2.100000,7.1085,0.0,0.0,10.495713,1009.000000,0.0,0.00


In [6]:
taxi_zone_lookup= pd.read_csv('/content/drive/My Drive/taxi_zone_lookup.csv')
pickup_lookup = taxi_zone_lookup.rename(columns={
    'LocationID': 'PULocationID',
    'Borough': 'pickup_borough',
    'Zone': 'pickup_zone'
})

dropoff_lookup = taxi_zone_lookup.rename(columns={
    'LocationID': 'DOLocationID',
    'Borough': 'dropoff_borough',
    'Zone': 'dropoff_zone'
})

merged_df = pd.merge(merged_df, pickup_lookup[['PULocationID', 'pickup_borough', 'pickup_zone']], on='PULocationID', how='left').drop(['PULocationID'], axis=1)
merged_df = pd.merge(merged_df, dropoff_lookup[['DOLocationID', 'dropoff_borough', 'dropoff_zone']], on='DOLocationID', how='left').drop(['DOLocationID'], axis=1)

merged_df

,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,payment_type,fare_amount,tip_amount,total_amount,congestion_surcharge,airport_fee,...,rain,precipitation,wind_speed_10m,pressure_msl,snowfall,snow_depth,pickup_borough,pickup_zone,dropoff_borough,dropoff_zone
0,2019-01-09 22:54:47,2019-01-09 23:23:09,1.0,6.20,1,23.0,2.00,26.30,NaN,0,...,0.1,0.1,23.110207,1015.500000,0.0,0.01,Manhattan,West Village,Queens,Old Astoria
1,2019-01-11 21:30:16,2019-01-11 21:53:14,2.0,4.55,2,19.5,0.00,20.80,NaN,0,...,0.0,0.0,20.858265,1012.500000,0.0,0.01,Manhattan,Greenwich Village South,Manhattan,Lincoln Square East
2,2019-01-06 19:06:07,2019-01-06 19:23:40,1.0,2.60,1,13.0,2.50,16.30,NaN,0,...,0.0,0.0,5.447788,1029.400024,0.0,0.00,Manhattan,Midtown North,Manhattan,West Village
3,2019-01-12 04:23:27,2019-01-12 04:39:41,1.0,4.37,1,16.0,3.46,20.76,NaN,0,...,0.0,0.0,16.735161,1014.299988,0.0,0.01,Manhattan,Union Sq,Brooklyn,Williamsburg (South Side)
4,2019-01-31 11:46:57,2019-01-31 11:55:58,1.0,0.89,1,7.5,1.66,9.96,0.0,0,...,0.0,0.0,7.568566,1001.200012,0.0,0.01,Manhattan,Flatiron,Manhattan,West Chelsea/Hudson Yards
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11995,2019-12-22 21:15:14,2019-12-22 21:20:21,2.0,0.77,1,5.5,1.40,10.70,2.5,0,...,0.0,0.0,12.768586,990.799988,0.0,0.00,Manhattan,Greenwich Village North,Manhattan,Gramercy
11996,2019-12-02 08:48:54,2019-12-02 09:00:47,1.0,2.20,1,10.5,1.00,12.30,0.0,0,...,0.0,0.0,21.252199,1018.700012,0.0,0.00,Brooklyn,Cobble Hill,Brooklyn,Gowanus
11997,2019-12-03 16:01:29,2019-12-03 16:09:51,1.0,1.20,1,7.5,1.20,13.00,2.5,0,...,0.0,0.0,12.496718,1025.400024,0.0,0.00,Manhattan,Murray Hill,Manhattan,Gramercy
11998,2019-12-24 13:17:01,2019-12-24 13:19:07,1.0,0.48,1,3.5,1.36,8.16,2.5,0,...,0.0,0.0,10.495713,1009.000000,0.0,0.00,Manhattan,Flatiron,Manhattan,Flatiron


In [7]:
holidays_df = pd.read_csv("/content/drive/My Drive/publicholiday.US.2019.csv")
holidays_df['Date'] = pd.to_datetime(holidays_df['Date']).dt.date
merged_df['is_holiday'] = merged_df['pickup_date'].isin(holidays_df['Date'])
merged_df.drop(['pickup_date'], axis= 'columns')
merged_df

,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,payment_type,fare_amount,tip_amount,total_amount,congestion_surcharge,airport_fee,...,precipitation,wind_speed_10m,pressure_msl,snowfall,snow_depth,pickup_borough,pickup_zone,dropoff_borough,dropoff_zone,is_holiday
0,2019-01-09 22:54:47,2019-01-09 23:23:09,1.0,6.20,1,23.0,2.00,26.30,NaN,0,...,0.1,23.110207,1015.500000,0.0,0.01,Manhattan,West Village,Queens,Old Astoria,False
1,2019-01-11 21:30:16,2019-01-11 21:53:14,2.0,4.55,2,19.5,0.00,20.80,NaN,0,...,0.0,20.858265,1012.500000,0.0,0.01,Manhattan,Greenwich Village South,Manhattan,Lincoln Square East,False
2,2019-01-06 19:06:07,2019-01-06 19:23:40,1.0,2.60,1,13.0,2.50,16.30,NaN,0,...,0.0,5.447788,1029.400024,0.0,0.00,Manhattan,Midtown North,Manhattan,West Village,False
3,2019-01-12 04:23:27,2019-01-12 04:39:41,1.0,4.37,1,16.0,3.46,20.76,NaN,0,...,0.0,16.735161,1014.299988,0.0,0.01,Manhattan,Union Sq,Brooklyn,Williamsburg (South Side),False
4,2019-01-31 11:46:57,2019-01-31 11:55:58,1.0,0.89,1,7.5,1.66,9.96,0.0,0,...,0.0,7.568566,1001.200012,0.0,0.01,Manhattan,Flatiron,Manhattan,West Chelsea/Hudson Yards,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11995,2019-12-22 21:15:14,2019-12-22 21:20:21,2.0,0.77,1,5.5,1.40,10.70,2.5,0,...,0.0,12.768586,990.799988,0.0,0.00,Manhattan,Greenwich Village North,Manhattan,Gramercy,False
11996,2019-12-02 08:48:54,2019-12-02 09:00:47,1.0,2.20,1,10.5,1.00,12.30,0.0,0,...,0.0,21.252199,1018.700012,0.0,0.00,Brooklyn,Cobble Hill,Brooklyn,Gowanus,False
11997,2019-12-03 16:01:29,2019-12-03 16:09:51,1.0,1.20,1,7.5,1.20,13.00,2.5,0,...,0.0,12.496718,1025.400024,0.0,0.00,Manhattan,Murray Hill,Manhattan,Gramercy,False
11998,2019-12-24 13:17:01,2019-12-24 13:19:07,1.0,0.48,1,3.5,1.36,8.16,2.5,0,...,0.0,10.495713,1009.000000,0.0,0.00,Manhattan,Flatiron,Manhattan,Flatiron,False


In [8]:
import requests

url = "https://api.census.gov/data/2019/acs/acs5"
params = {
    "get": "NAME,B19013_001E",
    "for": "zip code tabulation area:*",
    "in": "state:36",
    "key": "ba037a99c1b6be8526b850c03ebd58819c88ff0c"
}

response = requests.get(url, params=params)
data = response.json()

zipcode_df = pd.DataFrame(data[1:], columns=data[0])
zipcode_df.rename(columns={
    "B19013_001E": "median_household_income",
    "zip code tabulation area": "zipcode"
}, inplace=True)

zipcode_df["median_household_income"] = pd.to_numeric(zipcode_df["median_household_income"], errors="coerce")
zipcode_df= zipcode_df.drop(['NAME', 'state'], axis= 'columns')
zipcode_df

,median_household_income,zipcode
0,112946,11804
1,102639,11930
2,123125,11941
3,113182,11962
4,130337,11702
...,...,...
1789,48387,13305
1790,50373,13321
1791,51429,13345
1792,-666666666,13352


In [18]:
import geopandas as gpd

taxi_zones = gpd.read_file('/content/drive/My Drive/taxi_zones/taxi_zones.shp')
zctas = gpd.read_file('/content/drive/My Drive/cb_2018_us_zcta510_500k/cb_2018_us_zcta510_500k.shp')
zctas = zctas[zctas['ZCTA5CE10'].str.startswith(('100', '101', '102', '103', '104'))]
zctas = zctas.to_crs(taxi_zones.crs)

zones_with_zip = gpd.sjoin(taxi_zones, zctas, how='left', predicate='intersects')
zones_with_zip = zones_with_zip[['LocationID', 'zone', 'borough', 'ZCTA5CE10']]
zones_with_zip.rename(columns={'ZCTA5CE10': 'zipcode'}, inplace=True)
zones_with_zip

,LocationID,zone,borough,zipcode
0,1,Newark Airport,EWR,NaN
1,2,Jamaica Bay,Queens,NaN
2,3,Allerton/Pelham Gardens,Bronx,10461
2,3,Allerton/Pelham Gardens,Bronx,10475
2,3,Allerton/Pelham Gardens,Bronx,10469
...,...,...,...,...
261,262,Yorkville East,Manhattan,10128
261,262,Yorkville East,Manhattan,10029
262,263,Yorkville West,Manhattan,10075
262,263,Yorkville West,Manhattan,10028


In [22]:
zones_with_income = pd.merge(zones_with_zip, zipcode_df, on='zipcode', how='left')
zones_with_income

,LocationID,zone,borough,zipcode,median_household_income
0,1,Newark Airport,EWR,NaN,NaN
1,2,Jamaica Bay,Queens,NaN,NaN
2,3,Allerton/Pelham Gardens,Bronx,10461,58305.0
3,3,Allerton/Pelham Gardens,Bronx,10475,51951.0
4,3,Allerton/Pelham Gardens,Bronx,10469,65399.0
...,...,...,...,...,...
629,262,Yorkville East,Manhattan,10128,121829.0
630,262,Yorkville East,Manhattan,10029,33720.0
631,263,Yorkville West,Manhattan,10075,138640.0
632,263,Yorkville West,Manhattan,10028,122969.0


In [23]:
zip_lat_lng = pd.read_csv('/content/drive/My Drive/zipCodeToLatLong.csv')
zip_lat_lng['zip'] = zip_lat_lng['zip'].astype(str).str.zfill(5)
zip_lat_lng

,zip,latitude,longitude
0,00601,18.180555,-66.749961
1,00602,18.361945,-67.175597
2,00603,18.458497,-67.123906
3,00606,18.158327,-66.932928
4,00610,18.294032,-67.127156
...,...,...,...
33786,99923,56.000518,-130.037474
33787,99925,55.550203,-132.945947
33788,99926,55.138352,-131.470425
33789,99927,56.331881,-133.606420


In [24]:
zones_with_income['zipcode'] = zones_with_income['zipcode'].astype(str).str.zfill(5)
zip_lat_lng.rename(columns=lambda x: x.strip().lower(), inplace=True)
zip_lat_lng.rename(columns={'zip': 'zipcode'}, inplace=True)

zip_pins_df = pd.merge(zones_with_income, zip_lat_lng, on='zipcode', how='left')
zip_pins_df.rename(columns={'latitude': 'latitude', 'longitude': 'longitude'}, inplace=True)

zip_pins_df = zip_pins_df[['zone', 'borough', 'zipcode', 'median_household_income', 'latitude', 'longitude']]
zip_pins_df

,zone,borough,zipcode,median_household_income,latitude,longitude
0,Newark Airport,EWR,00nan,NaN,NaN,NaN
1,Jamaica Bay,Queens,00nan,NaN,NaN,NaN
2,Allerton/Pelham Gardens,Bronx,10461,58305.0,40.846741,-73.840788
3,Allerton/Pelham Gardens,Bronx,10475,51951.0,40.875635,-73.826934
4,Allerton/Pelham Gardens,Bronx,10469,65399.0,40.868509,-73.846691
...,...,...,...,...,...,...
629,Yorkville East,Manhattan,10128,121829.0,40.781498,-73.950199
630,Yorkville East,Manhattan,10029,33720.0,40.792249,-73.947333
631,Yorkville West,Manhattan,10075,138640.0,40.773413,-73.956291
632,Yorkville West,Manhattan,10028,122969.0,40.776447,-73.953383


In [26]:
zip_lookup = (
    zip_pins_df
    .drop_duplicates(subset=['zone', 'borough'], keep='first')
    .set_index(['zone', 'borough'])
    [['zipcode', 'median_household_income', 'latitude', 'longitude']]
)

df = (merged_df.merge(
        zip_lookup,
        how='left',
        left_on=['pickup_zone', 'pickup_borough'],
        right_index=True,
        validate='many_to_one',
        suffixes=('', '_pickup')
    )
)

df = (df.merge(
        zip_lookup,
        how='left',
        left_on=['dropoff_zone', 'dropoff_borough'],
        right_index=True,
        validate='many_to_one',
        suffixes=('', '_dropoff')
    )
)

df = df.rename(columns={
    'zipcode':'pickup_zipcode',
    'median_household_income':'pickup_income',
    'latitude':'pickup_lat',
    'longitude':'pickup_lng',
    'zipcode_dropoff':'dropoff_zipcode',
    'median_household_income_dropoff':'dropoff_income',
    'latitude_dropoff':'dropoff_lat',
    'longitude_dropoff':'dropoff_lng',
})

mask_geo = df[['pickup_lat', 'dropoff_lat']].isna().any(axis=1)
mask_inc = (df['pickup_income'] <= 0) | (df['dropoff_income'] <= 0)
df = df.fillna(0)
clean_df = df[~(mask_geo | mask_inc)].copy().drop(['tpep_pickup_datetime', 'tpep_dropoff_datetime'], axis=1)
clean_df

,passenger_count,trip_distance,payment_type,fare_amount,tip_amount,total_amount,congestion_surcharge,airport_fee,pickup_date,pickup_time,...,dropoff_zone,is_holiday,pickup_zipcode,pickup_income,pickup_lat,pickup_lng,dropoff_zipcode,dropoff_income,dropoff_lat,dropoff_lng
1,2.0,4.55,2,19.5,0.00,20.80,0.0,0,2019-01-11,21:30:16,...,Lincoln Square East,False,10012,106467.0,40.725625,-73.997936,10019,103792.0,40.765735,-73.985065
2,1.0,2.60,1,13.0,2.50,16.30,0.0,0,2019-01-06,19:06:07,...,West Village,False,10019,103792.0,40.765735,-73.985065,10014,133501.0,40.734145,-74.006026
4,1.0,0.89,1,7.5,1.66,9.96,0.0,0,2019-01-31,11:46:57,...,West Chelsea/Hudson Yards,False,10014,133501.0,40.734145,-74.006026,10014,133501.0,40.734145,-74.006026
5,1.0,1.90,1,10.5,2.00,13.80,0.0,0,2019-01-17,20:32:50,...,East Village,False,10014,133501.0,40.734145,-74.006026,10002,36982.0,40.715969,-73.986895
6,1.0,0.30,2,4.5,0.00,5.30,0.0,0,2019-01-07,14:05:25,...,Upper West Side South,False,10023,132605.0,40.774567,-73.979016,10023,132605.0,40.774567,-73.979016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11993,1.0,2.34,1,11.5,2.96,17.76,2.5,0,2019-12-19,07:52:08,...,Upper East Side South,False,10018,122484.0,40.755327,-73.992696,10022,150718.0,40.758570,-73.967930
11995,2.0,0.77,1,5.5,1.40,10.70,2.5,0,2019-12-22,21:15:14,...,Gramercy,False,10014,133501.0,40.734145,-74.006026,10009,63717.0,40.726392,-73.978604
11997,1.0,1.20,1,7.5,1.20,13.00,2.5,0,2019-12-03,16:01:29,...,Gramercy,False,10016,126628.0,40.745182,-73.978053,10009,63717.0,40.726392,-73.978604
11998,1.0,0.48,1,3.5,1.36,8.16,2.5,0,2019-12-24,13:17:01,...,Flatiron,False,10014,133501.0,40.734145,-74.006026,10014,133501.0,40.734145,-74.006026


In [31]:
clean_df.to_csv('data.csv', index=False)